# **II. TRIỂN KHAI THUẬT TOÁN LINEAR REGRESSION**

**1. Đọc dữ liệu**

In [ ]:
import re
import pandas as pd
import numpy as np

In [ ]:
data = []
with open("/content/drive/MyDrive/Colab Notebooks/DSLAB TRAINING/Data/DeathRate.txt") as f:
  dataLineList = f.read().splitlines()
  for i in range(len(dataLineList)):
    dataLineList[i] = re.sub(' +', ' ', dataLineList[i])
    dataLineList[i] = dataLineList[i].split(' ')[1:]
    data.append(dataLineList[i])

In [ ]:
data = pd.DataFrame(data)
data.index = np.arange(1, len(data) + 1)
data.columns = np.append(['A' + str(i) for i in range(1, 16)], ['B'])
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,B
1,36,27,71,8.1,3.34,11.4,81.5,3243,8.8,42.6,11.7,21,15,59,59,921.870
2,35,23,72,11.1,3.14,11.0,78.8,4281,3.6,50.7,14.4,8,10,39,57,997.875
3,44,29,74,10.4,3.21,9.8,81.6,4260,0.8,39.4,12.4,6,6,33,54,962.354
4,47,45,79,6.5,3.41,11.1,77.5,3125,27.1,50.2,20.6,18,8,24,56,982.291
5,43,35,77,7.6,3.44,9.6,84.6,6441,24.4,43.7,14.3,43,38,206,55,1071.289


In [ ]:
# Convert to true dataType

data = data.apply(pd.to_numeric)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 1 to 60
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      60 non-null     int64  
 1   A2      60 non-null     int64  
 2   A3      60 non-null     int64  
 3   A4      60 non-null     float64
 4   A5      60 non-null     float64
 5   A6      60 non-null     float64
 6   A7      60 non-null     float64
 7   A8      60 non-null     int64  
 8   A9      60 non-null     float64
 9   A10     60 non-null     float64
 10  A11     60 non-null     float64
 11  A12     60 non-null     int64  
 12  A13     60 non-null     int64  
 13  A14     60 non-null     int64  
 14  A15     60 non-null     int64  
 15  B       60 non-null     float64
dtypes: float64(8), int64(8)
memory usage: 8.0 KB


**2. Chuẩn hóa dữ liệu**

In [ ]:
def normalize_and_add_ones(X):
  X = np.array(X)
  X_max = np.array([[np.amax(X[:, column_id])
                      for column_id in range(X.shape[1])]
                      for _ in range(X.shape[0])])
  X_min = np.array([[np.amin(X[:, column_id])
                      for column_id in range(X.shape[1])]
                      for _ in range(X.shape[0])])
  X_normalized = (X - X_min) / (X_max - X_min)
  ones = np.array([[1] for _ in range(X_normalized.shape[0])])
  return np.column_stack((ones, X_normalized))

In [ ]:
data = normalize_and_add_ones(data)
data = pd.DataFrame(data)
data.index = np.arange(1, len(data) + 1)
data.columns = np.append(['A' + str(i) for i in range(0, 16)], ['B'])
data.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,B
1,1.0,0.52,0.211268,0.363636,0.403226,0.688525,0.727273,0.615063,0.218213,0.212202,0.339768,0.135294,0.030912,0.044025,0.209386,0.600000,0.406723
2,1.0,0.50,0.154930,0.409091,0.887097,0.360656,0.606061,0.502092,0.343909,0.074271,0.652510,0.294118,0.010819,0.028302,0.137184,0.542857,0.642454
3,1.0,0.68,0.239437,0.500000,0.774194,0.475410,0.242424,0.619247,0.341366,0.000000,0.216216,0.176471,0.007728,0.015723,0.115523,0.457143,0.532285
4,1.0,0.74,0.464789,0.727273,0.145161,0.803279,0.636364,0.447699,0.203923,0.697613,0.633205,0.658824,0.026275,0.022013,0.083032,0.514286,0.594120
5,1.0,0.66,0.323944,0.636364,0.322581,0.852459,0.181818,0.744770,0.605473,0.625995,0.382239,0.288235,0.064915,0.116352,0.740072,0.485714,0.870149


**3. Triển khai mô hình**

In [ ]:
# Xây dựng lớp RidgeRegression

class RidgeRegression:
  def __init__(self):
    return

  def fit(self, X_train, Y_train, LAMBDA):
    assert len(X_train.shape) == 2 and X_train.shape[0] == Y_train.shape[0]
    W = np.linalg.inv(X_train.transpose().dot(X_train) + LAMBDA * np.identity(X_train.shape[1])).dot(X_train.transpose()).dot(Y_train)
    return W

  def fit_gradient_descent(self, X_train, Y_train, LAMBDA, learning_rate, max_num_epoch=100, batch_size=128):
        W = np.random.randn(X_train.shape[1])
        last_loss = 10e+8
        for ep in range(max_num_epoch):
            arr = np.array(range(X_train.shape[0]))
            np.random.shuffle(arr)
            X_train = X_train[arr]
            Y_train = Y_train[arr]
            total_minibatch = int(np.ceil(X_train.shape[0] / batch_size))
            for i in range(total_minibatch):
                index = i * batch_size
                X_train_sub = X_train[index:index + batch_size]
                Y_train_sub = Y_train[index:index + batch_size]
                grad = X_train_sub.T.dot(X_train_sub.dot(W) - Y_train_sub) + LAMBDA * W
                W = W - learning_rate * grad
            new_loss = self.compute_RSS(self.predict(W, X_train), Y_train)
            if (np.abs(new_loss - last_loss) <= 1e-5):
                break
            last_loss = new_loss
        return W
    
  def predict(self, W, X_new):
    X_new = np.array(X_new)
    Y_new = X_new.dot(W)
    return Y_new

  def compute_RSS(self, Y_new, Y_predicted):
    loss = 1. / Y_new.shape[0] * np.sum((Y_new - Y_predicted) ** 2)
    return loss

  def get_the_best_LAMBDA(self, X_train, Y_train):
    def cross_validation(num_folds, LAMBDA):
      row_ids = np.array(range(X_train.shape[0]))
      valid_ids = np.split(row_ids[: len(row_ids) - len(row_ids) % num_folds], num_folds)
      valid_ids[-1] = np.append(valid_ids[-1], row_ids[len(row_ids) - len(row_ids) % num_folds:])
      train_ids = [[k for k in row_ids if k not in valid_ids[i]] for i in range(num_folds)]
      aver_RSS = 0
      for i in range(num_folds):
        valid_part = {'X': X_train[valid_ids[i]], 'Y': Y_train[valid_ids[i]]}
        train_part = {'X': X_train[train_ids[i]], 'Y': Y_train[train_ids[i]]}
        W = self.fit(train_part['X'], train_part['Y'], LAMBDA)
        Y_predicted = self.predict(W, valid_part['X'])
        aver_RSS += self.compute_RSS(valid_part['Y'], Y_predicted)
      return aver_RSS / num_folds
    
    def range_scan(best_LAMBDA, minimum_RSS, LAMBDA_values):
      for current_LAMBDA in LAMBDA_values:
        aver_RSS = cross_validation(num_folds=5, LAMBDA=current_LAMBDA)
        if aver_RSS < minimum_RSS:
          best_LAMBDA = current_LAMBDA
          minimum_RSS = aver_RSS
      return best_LAMBDA, minimum_RSS

    best_LAMBDA, minimum_RSS = range_scan(best_LAMBDA=0, minimum_RSS=10000 ** 2, LAMBDA_values= range(50))
    LAMBDA_values = [k * 1. / 1000 for k in range(max(0, (best_LAMBDA - 1) * 1000), (best_LAMBDA + 1) * 1000)] # step_size = 0.001
    best_LAMBDA, minimum_RSS = range_scan(best_LAMBDA=best_LAMBDA, minimum_RSS=minimum_RSS, LAMBDA_values= LAMBDA_values)
    return best_LAMBDA

In [ ]:
X_train, Y_train = data.iloc[:50, :data.shape[1] - 1].values, data.iloc[:50, data.shape[1] - 1].values
X_test, Y_test = data.iloc[50:, :data.shape[1] - 1].values, data.iloc[50:, data.shape[1] - 1].values

ridge_regression = RidgeRegression()
best_LAMBDA = ridge_regression.get_the_best_LAMBDA(X_train, Y_train)
print('Best LAMBDA: ', best_LAMBDA)
W_learned = ridge_regression.fit(X_train, Y_train, best_LAMBDA)
Y_predicted = ridge_regression.predict(W_learned, X_test)
print('RSS: ', ridge_regression.compute_RSS(Y_test, Y_predicted))

Best LAMBDA:  0.861
RSS:  0.01612552348560164


In [ ]:
# Using sklearn

from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error

clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])
clf.fit(X_train, Y_train)
Y_predicted = clf.predict(X_test)
print('RSS: ', mean_squared_error(Y_test, Y_predicted))

RSS:  0.016751436122482987
